# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [30]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [31]:

cities_csv_path = "../WeatherPy/cities.csv"

# Import the data into a Pandas DataFrame
cities_df = pd.read_csv(cities_csv_path, index_col='Unnamed: 0')
cities_df.describe()


,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,4.750000e+02
mean,19.403374,21.962114,16.417853,67.734737,58.185263,3.700674,1.619374e+09
std,33.994719,93.035287,11.915117,22.646177,36.117124,2.434209,2.958224e+02
min,-54.800000,-175.200000,-20.110000,4.000000,0.000000,0.150000,1.619373e+09
25%,-8.289150,-59.471500,8.330000,54.500000,20.000000,1.825000,1.619373e+09
50%,21.283300,24.662200,18.260000,72.000000,75.000000,3.130000,1.619374e+09
75%,49.172350,110.038650,26.000000,86.000000,90.000000,5.140000,1.619374e+09
max,78.218600,179.316700,40.310000,100.000000,100.000000,15.430000,1.619374e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [32]:

gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into locations
locations = cities_df[["Lat", "Lng"]].astype(float)

# Store 'Humidity' as list
humidity = cities_df["Humidity"].astype(float)


In [33]:
fig = gmaps.figure(zoom_level=2, center=(16.276737, 1.482970))

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100,
                                 point_radius=4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [34]:
weather_df = cities_df.loc[(cities_df["Wind Speed"] <10) & 
(cities_df["Cloudiness"] == 0) & (cities_df["Max Temp"] >20) &
(cities_df["Max Temp"] <26)].dropna()

weather_df.count()
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
31,lasa,46.6166,10.7002,23.00,35,0,3.09,IT,1619373364
44,catuday,16.2923,119.8062,25.01,75,0,4.70,PH,1619373480
136,santiago del estero,-27.7951,-64.2615,23.73,37,0,2.70,AR,1619373521
272,antofagasta,-23.6500,-70.4000,21.11,68,0,5.14,CL,1619373333
349,buin,-33.7333,-70.7500,24.00,28,0,2.57,CL,1619373894


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [39]:
hotel_df = weather_df


hotel_df["Hotel Name"] = ""
hotel_df

# Set parameters to search for hotels within 5000 meters
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    

    lat = row["Lat"]
    lng = row["Lng"]
    
    # Change location each iteration while leaving origional params in place
    params["location"] = f"{lat},{lng}"
    
    # URL
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    hotel_name = requests.get(base_url, params=params)
    
    # Convert to json
    hotel_name = hotel_name.json()
    
    # try-except to skip any missing data
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except:
        print("Missin field/result ... skipping")


In [40]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
31,lasa,46.6166,10.7002,23.00,35,0,3.09,IT,1619373364,Pension Feldgärtenhof
44,catuday,16.2923,119.8062,25.01,75,0,4.70,PH,1619373480,GResort
136,santiago del estero,-27.7951,-64.2615,23.73,37,0,2.70,AR,1619373521,Apartamentos Castelli 90
272,antofagasta,-23.6500,-70.4000,21.11,68,0,5.14,CL,1619373333,Holiday Inn Express Antofagasta
349,buin,-33.7333,-70.7500,24.00,28,0,2.57,CL,1619373894,Paihuen
377,karak,31.1667,35.7500,25.00,44,0,5.14,JO,1619373934,Cairwan Hotel
447,san luis,-33.2950,-66.3356,20.88,42,0,2.66,AR,1619374028,Huarpes Hotel


In [41]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [42]:
marker_layer = gmaps.marker_layer(locations, info_box_content= hotel_info)

fig = gmaps.figure(zoom_level=2, center=(16.276737, 1.482970))
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))